# Instacart Market Basket Analysis

The goal of this project is to gain comprehensive insights into customer behavior, product performance, and purchasing patterns using transactional data. The project will be divided into several key stages, including data exploration and preprocessing, exploratory data analysis (EDA), and business questions and analysis. The following are the detailed requirements:

### Download data

In [1]:
!mkdir data
%cd data

/content/data


In [2]:
import gdown

urls = [
    'https://drive.google.com/uc?id=1hjEu5fQBVOkE-YmKL0UN2HXSzz_OBFDr&export=download',
    'https://drive.google.com/uc?id=1UIJXjSh8349OuMH083mTqhKMcXc0QHY8&export=download',
    'https://drive.google.com/uc?id=1qrYbltugl76RWcIiqIl8ElHiKAkTZnwD&export=download',
    'https://drive.google.com/uc?id=1aV1dQTPQwyA2xMSZfFmgPwgGFc-dwEWX&export=download',
    'https://drive.google.com/uc?id=17t5qeApHrO3EeXvYV_PZ-D_1SETj08P0&export=download',
    'https://drive.google.com/uc?id=16aLaJJOY54fcobDsajP-Fu7yxhsrRSSn&export=download'
]

# Download all files using a list comprehension
downloads = [gdown.download(url, quiet=False) for url in urls]

Downloading...
From (original): https://drive.google.com/uc?id=1hjEu5fQBVOkE-YmKL0UN2HXSzz_OBFDr&export=download
From (redirected): https://drive.google.com/uc?id=1hjEu5fQBVOkE-YmKL0UN2HXSzz_OBFDr&export=download&confirm=t&uuid=bb2b9571-b433-42b9-b60d-ce868e39592b
To: /content/data/order_products__prior.csv
100%|██████████| 578M/578M [00:12<00:00, 46.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1UIJXjSh8349OuMH083mTqhKMcXc0QHY8&export=download
From (redirected): https://drive.google.com/uc?id=1UIJXjSh8349OuMH083mTqhKMcXc0QHY8&export=download&confirm=t&uuid=995f13cf-fa67-4f24-a313-e205ac0db0fb
To: /content/data/orders.csv
100%|██████████| 109M/109M [00:02<00:00, 47.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qrYbltugl76RWcIiqIl8ElHiKAkTZnwD&export=download
To: /content/data/order_products__train.csv
100%|██████████| 24.7M/24.7M [00:00<00:00, 90.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aV1dQTPQwyA2xMSZfFmgPwgGFc-dwEWX&export=dow

In [3]:
%cd ..

/content


## 1. Data Exploration and Preprocessing
a. Load the Data:
- Load the CSV files into pandas DataFrames.

In [4]:
import pandas as pd

# Load the CSV files into pandas DataFrames
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
order_products_prior = pd.read_csv('data/order_products__prior.csv')
order_products_train = pd.read_csv('data/order_products__train.csv')
aisles = pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')

- Check for missing values and data types.

In [5]:
orders.isnull().sum()

,0
order_id,0
user_id,0
eval_set,0
order_number,0
order_dow,0
order_hour_of_day,0
days_since_prior_order,206209


In [6]:
# Create a list of DataFrames
dfs = [orders, products, order_products_prior, order_products_train, aisles, departments]
df_names = ['Orders', 'Products', 'Order Products Prior', 'Order Products Train', 'Aisles', 'Departments']

# Create a dictionary to hold the missing values information
missing_values_dict = {'DataFrame': [], 'Column': [], 'Missing Values': []}

# Check for missing values in each DataFrame
for df, name in zip(dfs, df_names):
    missing_values = df.isnull().sum()
    for column, missing in missing_values.items():
        missing_values_dict['DataFrame'].append(name)
        missing_values_dict['Column'].append(column)
        missing_values_dict['Missing Values'].append(missing)

# Convert the dictionary to a DataFrame
missing_values_df = pd.DataFrame(missing_values_dict)

In [7]:
missing_values_df

,DataFrame,Column,Missing Values
0,Orders,order_id,0
1,Orders,user_id,0
2,Orders,eval_set,0
3,Orders,order_number,0
4,Orders,order_dow,0
5,Orders,order_hour_of_day,0
6,Orders,days_since_prior_order,206209
7,Products,product_id,0
8,Products,product_name,0
9,Products,aisle_id,0


b. Data Cleaning:
- Handle missing values.

In [8]:
# Handling missing values
# Here we'll fill missing values with placeholders
orders.fillna({'days_since_prior_order': 0}, inplace=True)

- Merge DataFrames as needed (e.g., joining order_products with products).

In [ ]:
#concat train order and prior orders
order_products_all = pd.concat([order_products_prior, order_products_train]).sort_values(by=['order_id'])

#merge with rest
order_products_all = pd.merge(left = order_products_all, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
order_products_all = pd.merge(left = order_products_all, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
order_products_all = pd.merge(left = order_products_all, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
order_products_all = pd.merge(left = order_products_all, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

col_order = ['user_id','order_id','product_id','aisle_id','department_id','add_to_cart_order',
 'reordered','product_name','aisle','department','eval_set','order_number','order_dow','order_hour_of_day',
 'days_since_prior_order']

order_products_all = order_products_all[col_order]

In [ ]:
order_products_all.head()

c. Basic Descriptive Statistics:
- Calculate summary statistics for numerical features.

In [ ]:
order_products_all['user_id'].nunique()

In [ ]:
# Summary statistics for numerical features in the order_products_all DataFrame
print("\nSummary Statistics for All Order Products DataFrame:")
order_products_all.describe()

In [ ]:
order_products_all.info()

In [ ]:
order_products_all.describe(include='object')

- Examine the distribution of categorical features.

In [ ]:
# Distribution of categorical features in the order_products_all_merged DataFrame
print("\nDistribution of product_id in All Order Products Merged DataFrame:")
order_products_all['product_name'].value_counts().head()

In [ ]:
print("\nDistribution of aisle_id in All Order Products Merged DataFrame:")
order_products_all['aisle_id'].value_counts().head()

In [ ]:
print("\nDistribution of department_id in All Order Products Merged DataFrame:")
order_products_all['department_id'].value_counts().head()

## 2. Exploratory Data Analysis (EDA)
a. Customer Behavior:
- Average number of orders per user.

In [ ]:
# Calculate the number of orders per user
orders_per_user = orders.groupby('user_id').size()

# Calculate the average number of orders per user
average_orders_per_user = orders_per_user.mean()
print(f"Average number of orders per user: {average_orders_per_user:.2f}")


- Average time between orders for each user.

In [ ]:
average_days_between_orders = orders.groupby('user_id')['days_since_prior_order'].mean()
average_days_between_orders

- Number of orders placed by each customer.

In [ ]:
# Calculate the number of orders placed by each customer
orders_count_per_customer = orders.groupby('user_id').size()

# Display the number of orders for the first few customers
print("Number of orders placed by each customer (sample):")
print(orders_count_per_customer.head())

- Customer segments based on purchase frequency.

In [ ]:
# Define customer segments based on the number of orders
order_counts = orders_count_per_customer

# Create segments based on quantiles
segments = pd.qcut(order_counts, q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Merge the segments with the customer data
customer_segments = pd.DataFrame({
    'user_id': order_counts.index,
    'order_count': order_counts.values,
    'segment': segments
})

print("\nCustomer Segments based on Purchase Frequency:")
customer_segments.head()

b. Product Analysis:
- Identify most popular products by frequency.

In [ ]:
# Count the frequency of each product_id
product_frequency = order_products_all['product_id'].value_counts()

product_frequency_df = product_frequency.reset_index()
product_frequency_df.columns = ['product_id', 'frequency']

# Merge with products to get product names
product_frequency_df = product_frequency_df.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Sort by frequency and display the top products
top_products = product_frequency_df.sort_values(by='frequency', ascending=False)
print("\nMost Popular Products by Frequency:")
print(top_products.head())

- Determine average order size (number of items per order).

In [ ]:
# Calculate the number of items per order
order_sizes = order_products_all.groupby('order_id').size()

# Calculate the average order size
average_order_size = order_sizes.mean()
print(f"\nAverage Order Size (Number of Items per Order): {average_order_size:.2f}")

c. Temporal Patterns:
- Analyze orders by day of the week and hour of the day.

In [ ]:
orders_by_dow = order_products_all['order_dow'].value_counts()
orders_by_hour = order_products_all['order_hour_of_day'].value_counts()

orders_by_dow

In [ ]:
orders_by_hour

In [ ]:
# Determine if the order was on a weekend or weekday
order_products_all['is_weekend'] = order_products_all['order_dow'].apply(lambda x: x in [0, 6])  # 0 = Sunday, 6 = Saturday

In [ ]:
product_sales_by_day = order_products_all.groupby(['product_id', 'product_name', 'is_weekend'])['order_id'].count().reset_index()
product_sales_by_day

In [ ]:
popular_products_weekday = product_sales_by_day[product_sales_by_day['is_weekend'] == False].sort_values('order_id', ascending=False)
popular_products_weekday

In [ ]:
popular_products_weekend = product_sales_by_day[product_sales_by_day['is_weekend'] == True].sort_values('order_id', ascending=False)
popular_products_weekend

d. Basket Analysis:
- Identify most frequently co-purchased items.

In [ ]:
chunk = order_products_all.head(100000)

In [ ]:
# Create a new DataFrame with all possible pairs within each order
merged = pd.merge(chunk, chunk, on='order_id')

# Filter out self-pairs and sort product pairs
merged = merged[merged['product_id_x'] != merged['product_id_y']]
merged['product_pair'] = merged.apply(lambda row: tuple(sorted((row['product_id_x'], row['product_id_y']))), axis=1)

In [ ]:
# Group by product pairs and count occurrences
pair_counts = merged.groupby('product_pair').size().reset_index(name='frequency')

# Sort by frequency and return top 10
copurchased_pairs = pair_counts.sort_values('frequency', ascending=False)

copurchased_pairs.head(10)

- Products often bought together on weekends vs. weekdays.

In [ ]:
merged_weekend = merged[merged['is_weekend_x'] & merged['is_weekend_y']]
pair_counts_weekend = merged_weekend.groupby('product_pair').size().reset_index(name='frequency')

# Sort by frequency and return top 10
copurchased_pairs_weekend = pair_counts_weekend.sort_values('frequency', ascending=False)

copurchased_pairs_weekend.head()


In [ ]:
merged_weekday = merged[~(merged['is_weekend_x'] & merged['is_weekend_y'])]
pair_counts_weekday = merged_weekday.groupby('product_pair').size().reset_index(name='frequency')

# Sort by frequency and return top 10
copurchased_pairs_weekday = pair_counts_weekday.sort_values('frequency', ascending=False)

copurchased_pairs_weekday.head()


## 3. Business Questions and Analysis
a. Popular Products:
- Analyze sales distribution of top-selling products.

In [ ]:
order_products_all['product_id'].nunique()

In [ ]:
product_counts = order_products_all.groupby(['product_id', 'product_name']).size().sort_values(ascending=False).reset_index(name='count')
product_counts

- Identify top 5 products commonly added to the cart first.

In [ ]:
# Filter for products added first (add_to_cart_order == 1)
first_added_products = order_products_all[order_products_all['add_to_cart_order'] == 1]

# Count the frequency of each product
product_counts = first_added_products.groupby(['product_id', 'product_name']).size().reset_index(name='frequency')

# Sort by frequency and return top 5
top_5_products = product_counts.sort_values('frequency', ascending=False).head(5)
top_5_products

b. Reorder Behavior:
- Analyze the reordered column to understand repeat purchase behavior.
- Products reordered the most.

In [ ]:
# Count reordered products
reordered_products = order_products_all[order_products_all['reordered'] == 1]['product_id'].nunique()
total_products = order_products_all['product_id'].nunique()

# Calculate reorder rate
reorder_rate = reordered_products / total_products

print("Reorder rate:", reorder_rate)

# Identify top reordered products
top_reordered_products = order_products_all[order_products_all['reordered'] == 1].groupby('product_id').size().reset_index(name='reorder_count').sort_values('reorder_count', ascending=False)

top_reordered_products.head(5)

- Reorder behavior based on day of the week and days since prior order.

In [ ]:
# Filter for reordered products
reordered_data = order_products_all[order_products_all['reordered'] == 1]

# Calculate reorder rate by day of week
reordered_by_day = reordered_data.groupby(reordered_data['order_dow'])['product_id'].nunique() / reordered_data['product_id'].nunique()

# Calculate reorder rate by days since prior order
reordered_by_days_since_prior_order = reordered_data.groupby('days_since_prior_order')['product_id'].nunique() / reordered_data['product_id'].nunique()


In [ ]:
reordered_by_day.sort_values(ascending=False)

In [ ]:
reordered_by_days_since_prior_order.sort_values(ascending=False)

- How the number of items in the cart impacts the likelihood of reordering.

In [ ]:
# Group by order_id and count items in the cart
order_data = order_products_all.groupby('order_id').size().reset_index(name='cart_size')

# Merge with original DataFrame
merged_data = pd.merge(order_products_all, order_data, on='order_id')

# Calculate reorder rate by cart size
reordered_by_cart_size = merged_data[merged_data['reordered'] == 1].groupby('cart_size')['product_id'].nunique() / merged_data['product_id'].nunique()


In [ ]:
reordered_by_cart_size.sort_values(ascending=False)

d. Department and Aisle Analysis:
- Best-selling department and aisle breakdown.

In [ ]:
# Calculate total sales by department and aisle
sales_data = order_products_all.groupby(['department', 'aisle']).size().reset_index(name='total_sales')


In [ ]:
# Identify top-selling departments and aisles
top_departments = sales_data.groupby('department')['total_sales'].sum().nlargest(5)
top_departments

In [ ]:
top_aisles = sales_data.groupby('aisle')['total_sales'].sum().nlargest(5)
top_aisles

- What is the “produce” department? Break it down by aisle.

In [ ]:
# Filter for products in the "produce" department
produce_data = order_products_all[order_products_all['department'] == 'produce']

# Group by aisle and count sales
produce_by_aisle = produce_data.groupby('aisle').size().reset_index(name='total_sales')

# Sort by total sales
produce_by_aisle = produce_by_aisle.sort_values('total_sales', ascending=False)

produce_by_aisle

- Differences in purchasing behavior based on different departments or aisles.

In [ ]:
# Calculate average order size by department and aisle
average_order_size = order_products_all.groupby(['department', 'aisle']).size().reset_index(name='order_size')
average_order_size

In [ ]:
# Calculate reorder rate by department and aisle
reordered_data = order_products_all[order_products_all['reordered'] == 1]
reordered_data

In [ ]:
reorder_rate = reordered_data.groupby(['department', 'aisle']).size() / order_products_all.groupby(['department', 'aisle']).size()
reorder_rate

In [ ]:
# Analyze customer demographics by department and aisle
customer_demographics = order_products_all.groupby(['department', 'aisle']).agg({'user_id': 'nunique'})
customer_demographics

## Date-based Analytics

In [ ]:
orders_chunk = orders.head(1000)

In [ ]:
# Create a starting date for simulation
start_date = pd.to_datetime('2023-08-20')  # Choose a reference date

# Sort orders by user_id and order_number
orders_sorted = orders_chunk.sort_values(by=['user_id', 'order_number'])

# Initialize the simulated order_date column
orders_sorted['simulated_date'] = start_date

In [ ]:
# Generate simulated dates based on days_since_prior_order
for user_id, group in orders_sorted.head(1000).groupby('user_id'):
    group['simulated_date'] = start_date + pd.to_timedelta(group['days_since_prior_order'].fillna(0).cumsum(), unit='d')
    orders_sorted.loc[group.index, 'simulated_date'] = group['simulated_date']

In [ ]:
# Ensure that the simulated dates are updated correctly
orders_sorted['simulated_date'] = pd.to_datetime(orders_sorted['simulated_date'])

In [ ]:
orders_sorted.head()

- Explore seasonal trends or patterns in purchasing behavior.

In [ ]:
# Calculate quarterly sales
sales_by_quarter = orders_sorted.groupby(pd.Grouper(key='simulated_date', freq='QE'))['order_id'].count()
sales_by_quarter

- Months with higher order volumes.

In [ ]:
# Add a month column for aggregation
orders_sorted['month'] = orders_sorted['simulated_date'].dt.to_period('M')

# Calculate monthly order volumes
monthly_order_volumes = orders_sorted.groupby('month').size()

print("\nMonthly Order Volumes:")
print(monthly_order_volumes)

In [ ]:
orders_sorted['simulated_date'].describe()

- Identify customers who haven’t placed an order in the last 30 days.

In [ ]:
import datetime

# Calculate the current date
current_date = datetime.datetime.now()

# Identify customers who have placed orders in the last 30 days
recent_customers = orders_sorted[orders_sorted['simulated_date'] >= current_date - pd.Timedelta(days=30)]['user_id'].unique()

# Find all customers
all_customers = orders_sorted['user_id'].unique()

# Identify inactive customers
inactive_customers = set(all_customers) - set(recent_customers)

# Create DataFrame for inactive customers
inactive_customers_df = pd.DataFrame(list(inactive_customers), columns=['user_id'])

inactive_customers_df


- Percentage of customers who have churned in the past quarter.

In [ ]:
# This example assumes a quarter is 3 months
quarter_start_date = current_date - pd.DateOffset(months=3)

# Identify customers who have placed orders in the past quarter
recent_customers = orders_sorted[
    (orders_sorted['simulated_date'] >= quarter_start_date) &
    (orders_sorted['simulated_date'] <= current_date)
]['user_id'].unique()

# Find all customers
all_customers = orders_sorted['user_id'].unique()

# Identify churned customers
churned_customers = set(all_customers) - set(recent_customers)

# Calculate the percentage of churned customers
churned_customers_count = len(churned_customers)
total_customers_count = len(all_customers)
churn_rate_percentage = (churned_customers_count / total_customers_count) * 100

churn_rate_percentage


## Thank You!